In [1]:
require 'optim'
require 'io'
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

dl = require 'dataload'
cmd = torch.CmdLine()
dofile("Code/utils.lua")
dofile("Code/utilsNN.lua")

...Utils file loaded	


In [2]:
datapath = 'data/0-output/'

Tensor = torch.Tensor
LongTensor = torch.LongTensor
ByteTensor = torch.ByteTensor

use_cuda = false

thresh = 0
edim = 10
metric = 'f1'
inputs = loadMetadata(datapath .. "dqn_metadata.csv")                                               
stoplist = loadStopdata(datapath .. 'stopwordids.csv')
adapt = true
metric ='f1'
use_cuda = false
embeddingSize = 20

nepochs = 2
gamma= 0.
learning_rate = 0.1
epsilon = 1
stopwordlist = stopwords 
mem_size = 100
optimParams = { learningRate = learning_rate }

vocabSize, query_data = initialize_variables(inputs, 20, datapath, 5, 30, stoplist, thresh, use_cuda)

print("...query loaded")
model = buildModel(model, vocabSize, embeddingSize, metric, adapt, usecuda)

Running on a subset of 20 observations	


...query loaded	
Running LSTM model to learn f1	


Using adaptive regularization	


In [3]:
maskLayer = nn.MaskedSelect()

SKIP = 1
SELECT = 2

math.randomseed(420)
torch.manualSeed(420)
criterion = nn.MSECriterion()

if adapt then 
    criterion = nn.ParallelCriterion():add(nn.MSECriterion()):add(nn.BCECriterion())
end


if use_cuda then
    criterion = criterion:cuda()
    model = model:cuda()
end
params, gradParams = model:getParameters()

query_id = 1 
memory, rougeRecall, rougePrecision, rougeF1, qValues = forwardpass(
                query_data, query_id, model, epsilon, gamma, 
                metric, thresh, stopwordlist, adapt, use_cuda
)
fullmemory = memory
-- Stacking data
fullmemory = stackMemory(memory, fullmemory, mem_size, adapt, use_cuda)

In [4]:
-- xinput = fullmemory[1]
-- reward = fullmemory[2]
-- actions_in = fullmemory[3]

xinput = memory[1]
actions_in = memory[3]
reward = memory[2]:resize(20, 1)

In [5]:
xinput = {xinput[1]:double(), xinput[2]:double(), xinput[3]:double()}

In [6]:
xinput

{
  1 : DoubleTensor - size: 20x5
  2 : DoubleTensor - size: 20x3
  3 : DoubleTensor - size: 20x30
}


In [9]:
model:forget()

predTotal = model:forward(xinput)
predQ = predTotal[1]
predReg = predTotal[2]
print(model:get(1):get(1):get(1):get(3).module.step)
print(model:get(1):get(1):get(2):get(3).module.step)
print(model:get(1):get(1):get(3):get(3).module.step)
-- model:reset()
predQOnActions = maskLayer:forward({predQ, actions_in})

-- ones = torch.ones(predQ:size(1))

ones = torch.ones(predQ:size(1)):resize(predQ:size(1), 1)
lossf = criterion:forward({predQOnActions, predReg}, {reward, ones})

gradOutput = criterion:backward({predQOnActions, predReg}, {reward, ones})
gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput[1] )

-- predQ:size(), actions_in:size(), reward:size(), ones:size()
-- {xinput, gradMaskLayer, gradOutput}
-- print(model:get(1):get(1):get(1):get(3).module.step)
-- print(model:get(1):get(1):get(2):get(3).module.step)
-- print(model:get(1):get(1):get(3):get(3).module.step)
-- print(model)
model:backward(xinput, {gradMaskLayer[1], gradOutput[2]} )

31	
31	
31	


...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.ConcatTable:
In 1 module of nn.Sequential:
In 2 module of nn.ParallelTable:
In 3 module of nn.Sequential:
...iscojavierarceo/torch/install/share/lua/5.1/rnn/LSTM.lua:184: assertion failed!
stack traceback:
	[C]: in function 'assert'
	...iscojavierarceo/torch/install/share/lua/5.1/rnn/LSTM.lua:184: in function '_updateGradInput'
	...eo/torch/install/share/lua/5.1/rnn/AbstractRecurrent.lua:59: in function 'updateGradInput'
	...avierarceo/torch/install/share/lua/5.1/rnn/Sequencer.lua:121: in function <...avierarceo/torch/install/share/lua/5.1/rnn/Sequencer.lua:106>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:55: in function <...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:50>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lu

In [ ]:
for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
    xinput = xin[1]
    actions_in = xin[2]
    predTotal = model:forward(xinput)                
    
    predQ = predTotal[1]
    predReg = predTotal[2]
    print(xinput, predTotal, predQ:size(1))
    
    predQOnActions = maskLayer:forward({predQ, actions_in}) 

    ones = torch.ones(predQ:size(1)):resize(predQ:size(1), 1)
    lossf = criterion:forward({predQOnActions, predReg}, {reward, ones})
    
    gradOutput = criterion:backward({predQOnActions, predReg}, {reward, ones})
    gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput[1])
    
--     model:backward(xinput, {gradMaskLayer[1], gradOutput[1]} )
end

In [ ]:
for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
    xinput = xin[1]
    actions_in = xin[2]

    if use_cuda then
        maskLayer = nn.MaskedSelect():cuda()
        actions_in = torch.CudaByteTensor(#actions_in):copy(actions_in)
    end

    local function feval(params)
        gradParams:zero()
        if adapt then
            local predTotal = model:forward(xinput)                
            local predQ = predTotal[1]
            local predReg = predTotal[2]
            local predQOnActions = maskLayer:forward({predQ, actions_in}) 
            local ones = torch.ones(predQ:size(1)):resize(predQ:size(1))
            lossf = criterion:forward({predQOnActions, predReg}, {reward, ones})
            local gradOutput = criterion:backward({predQOnActions, predReg}, {reward, ones})
            local gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput[1])
            print(#gradMaskLayer[2], #gradOutput[1])
            model:backward(xinput, {gradMaskLayer[1], gradOutput[2]})
        else 
            local predQ = model:forward(xinput)
            local predQOnActions = maskLayer:forward({predQ, actions_in}) 
            lossf = criterion:forward(predQOnActions, reward)
            local gradOutput = criterion:backward(predQOnActions, reward)
            local gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput)
            print(#gradOutput)
            model:backward(xinput, gradMaskLayer[1])
        end 
        return lossf, gradParams
    end
 --- optim.rmsprop returns \theta, f(\theta):= loss function
 _, lossv  = optim.rmsprop(feval, params, optimParams)
end
print(lossv[1])

In [ ]:
loss = backProp(fullmemory, params, gradParams, optimParams, model, 
    criterion, batch_size, mem_size, adapt, use_cuda)

In [10]:
xinput

{


  1 : DoubleTensor - size: 20x5
  2 : DoubleTensor - size: 20x3
  3 : DoubleTensor - size: 20x30
}


In [ ]:
model = buildModel(model, vocabSize, embeddingSize, metric, adapt, usecuda)

In [ ]:
function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        predict = nn.Sequential()
                :add(nn.JoinTable(2))
                :add(nn.Tanh())
                :add(nn.Linear(embeddingSize * 3, 2))
    else
        predict = nn.Sequential()
                :add(nn.JoinTable(2))
                :add(nn.ReLU())
                :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Using adaptive regularization")
        local predictmodule = nn.Sequential()
                    :add(pmodule)

        local regmodel = nn.Sequential()
            :add(nn.JoinTable(2))
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local qmodel = nn.Sequential()
            :add(nn.JoinTable(2))
            :add(nn.Linear(embeddingSize * 3, 2))

        local final = nn.ConcatTable()
            :add(regmodel)
            :add(qmodel)

        nnmodel = final
        else
            pmodule:add(predict)
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

In [ ]:
model:forget()

predTotal = model:forward(xinput)
predQ = predTotal[1]
predReg = predTotal[2]

-- model:reset()
predQOnActions = maskLayer:forward({predQ, actions_in})

-- ones = torch.ones(predQ:size(1))
ones = torch.ones(predQ:size(1)):resize(predQ:size(1), 1)

lossf = criterion:forward({predQOnActions, predReg}, {reward, ones})

gradOutput = criterion:backward({predQOnActions, predReg}, {reward, ones})
gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput[1] )
model:backward(xinput, {gradMaskLayer[1], gradOutput[2]} )